# AIN501 - Deep learning
## Assignment 1 - CIFAR 10
### Tran Trong Hieu - MSE23185

#### I. Import and check GPU

In [7]:
# Import section
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Check GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#### II. Load dataset CIFAR-10

In [9]:
# Transform
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Batch size
batch_size_train = 64
batch_size_test = 100

# Load dataset CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size_train, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=batch_size_test, shuffle=False, num_workers=2)

# Define 10 classes of CIFAR-10 dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### III. Build model with informations:
1. Using CNN with ELU activation function
2. Loss fuction: Cross entropy loss
3. Optimizer: Adam
4. Early stop and check point

In [10]:
# CNN model with ELU activation fuction
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Sequential(
      nn.Conv2d(3, 64, 3, padding=1),
      nn.BatchNorm2d(64),
      nn.ELU(),
      nn.Conv2d(64, 64, 3, padding=1),
      nn.BatchNorm2d(64),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.3)
    )

    self.conv2 = nn.Sequential(
      nn.Conv2d(64, 128, 3, padding=1),
      nn.BatchNorm2d(128),
      nn.ELU(),
      nn.Conv2d(128, 128, 3, padding=1),
      nn.BatchNorm2d(128),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.4)
    )

    self.conv3 = nn.Sequential(
      nn.Conv2d(128, 256, 3, padding=1),
      nn.BatchNorm2d(256),
      nn.ELU(),
      nn.Conv2d(256, 256, 3, padding=1),
      nn.BatchNorm2d(256),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.5)
    )

    self.conv4 = nn.Sequential(
      nn.Conv2d(256, 512, 3, padding=1),
      nn.BatchNorm2d(512),
      nn.ELU(),
      nn.Conv2d(512, 512, 3, padding=1),
      nn.BatchNorm2d(512),
      nn.ELU(),
      nn.MaxPool2d(2),
      nn.Dropout(0.5)
    )

    self.fc = nn.Sequential(
      nn.Flatten(),
      nn.Linear(512 * 2 * 2, 512),
      nn.ELU(),
      nn.Dropout(0.5),
      nn.Linear(512, 10)
    )

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.fc(x)
    return x

model = CNN().to(device)

#### IV. Define optimizer, loss function and early stop

In [14]:
# Define optimizer, loss function and early stop
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

best_loss = float('inf')
early_stop_count = 0
patience = 7
save_path = 'cnn_model.pth'

#### V. Train model

In [16]:
# Training

epochs = 200

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
      images, labels = images.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")
    scheduler.step(epoch_loss)

    # Early stopping logic
    if epoch_loss < best_loss:
      best_loss = epoch_loss
      early_stop_count = 0
      torch.save(model.state_dict(), save_path)
      print("Saved Best Model")
    else:
      early_stop_count += 1
      if early_stop_count >= patience:
          print("Early stopping triggered")
          break

# Load best model
model.load_state_dict(torch.load(save_path))

Epoch [1/200], Loss: 0.9148
Saved Best Model
Epoch [2/200], Loss: 0.9070
Saved Best Model
Epoch [3/200], Loss: 0.9066
Saved Best Model
Epoch [4/200], Loss: 0.9023
Saved Best Model
Epoch [5/200], Loss: 0.8971
Saved Best Model
Epoch [6/200], Loss: 0.8983
Epoch [7/200], Loss: 0.8913
Saved Best Model
Epoch [8/200], Loss: 0.8841
Saved Best Model
Epoch [9/200], Loss: 0.8840
Saved Best Model
Epoch [10/200], Loss: 0.8823
Saved Best Model
Epoch [11/200], Loss: 0.8788
Saved Best Model
Epoch [12/200], Loss: 0.8728
Saved Best Model
Epoch [13/200], Loss: 0.8728
Epoch [14/200], Loss: 0.8658
Saved Best Model
Epoch [15/200], Loss: 0.8689
Epoch [16/200], Loss: 0.8636
Saved Best Model
Epoch [17/200], Loss: 0.8618
Saved Best Model
Epoch [18/200], Loss: 0.8586
Saved Best Model
Epoch [19/200], Loss: 0.8547
Saved Best Model
Epoch [20/200], Loss: 0.8566
Epoch [21/200], Loss: 0.8537
Saved Best Model
Epoch [22/200], Loss: 0.8486
Saved Best Model
Epoch [23/200], Loss: 0.8473
Saved Best Model
Epoch [24/200], Los

<All keys matched successfully>

#### VI. Evaluation model

In [17]:
# Evaluation
def evaluate(model, testloader):
    model.eval()
    total = 0
    correct = 0
    class_correct = [0] * 10
    class_total = [0] * 10
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1

    print(f"\nTest Loss: {total_loss / total:.6f}\n")
    for i in range(10):
        acc = 100 * class_correct[i] / class_total[i]
        print(f"Test Accuracy of {classes[i]:10}: {acc:.0f}% ({class_correct[i]}/{class_total[i]})")
    print(f"Test Accuracy (Overall): {100 * correct / total:.0f}% ({correct}/{total})")

evaluate(model, testloader)


Test Loss: 0.688480

Test Accuracy of plane     : 91% (914/1000)
Test Accuracy of car       : 96% (959/1000)
Test Accuracy of bird      : 88% (877/1000)
Test Accuracy of cat       : 78% (780/1000)
Test Accuracy of deer      : 95% (950/1000)
Test Accuracy of dog       : 87% (866/1000)
Test Accuracy of frog      : 96% (960/1000)
Test Accuracy of horse     : 95% (946/1000)
Test Accuracy of ship      : 96% (964/1000)
Test Accuracy of truck     : 96% (956/1000)
Test Accuracy (Overall): 92% (9172/10000)
